# How to set a prior given T=1 non-zero effect?

Assume one non-zero effect $T = 1$. We investigate SuSiE performance when we have a PVE such that SuSiE has certain amount of power e.g. PVE $ = 0.05$.

## Results

**- Summary: A larger prior results in a slightly higher power, FDR is always 0. The size of confidence sets is stable around 5 in the GSEA data.**

* Prior does not have a strong influence on the SuSiE performance given only one non-zero effect. But GSEA data has a larger size of confidence sets compared to GTEx and single cell data. 

In [12]:
dscout.summary[dscout.summary$pve==0.05,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
19,1,0.05,0.01,0.84,0,5.309524,1,0.5,0.9899010
85,1,0.05,0.02,0.88,0,5.545455,1,0.5,0.9807201
151,1,0.05,0.03,0.88,0,5.522727,1,0.5,0.9807201
217,1,0.05,0.05,0.88,0,5.522727,1,0.5,0.9807201
283,1,0.05,0.10,0.88,0,5.500000,1,0.5,0.9807201
349,1,0.05,0.20,0.88,0,5.500000,1,0.5,0.9807201
415,1,0.05,0.40,0.88,0,5.500000,1,0.5,0.9807201
481,1,0.05,0.50,0.88,0,5.500000,1,0.5,0.9807201
547,1,0.05,0.70,0.88,0,5.522727,1,0.5,0.9807201
613,1,0.05,0.90,0.88,0,5.522727,1,0.5,0.9807201


## Code details

In [1]:
dscout_Q2 = readRDS('gsea_Q2.rds')
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$sim_gaussian.output.file),]
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$susie_prior.output.file),]

In [2]:
dscout_df = data.frame(dscout_Q2$sim_gaussian.effect_num, dscout_Q2$sim_gaussian.pve, dscout_Q2$susie_prior.prior,
                       dscout_Q2$score.hit, dscout_Q2$score.signal_num, dscout_Q2$score.cs_medianSize,
                       dscout_Q2$score.top_hit, dscout_Q2$sim_gaussian.mean_corX, dscout_Q2$susie_prior.avg_purity)
names(dscout_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [3]:
hitsum.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, sum)
hitmean.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, mean)
power.summary = hitmean.summary
power.summary$power = power.summary$hit / power.summary$effect_num
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, sum)
fdr.summary$fdr = round(1 - hitsum.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
cs_num.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, meannonzero)
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve + prior, dscout_df, meannonzero)

In [4]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior,
                            power.summary$power, fdr.summary$fdr, 
                            setsize.summary$cs_size, cs_num.summary$cs_num, 
                            tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'prior','power', 'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [5]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0

In [6]:
dscout.summary = dscout.summary[dscout.summary$effect_num==1, ]